In [117]:
import requests
import pandas as pd
import math
import json

In [118]:
df = pd.read_csv('demo.csv', index_col=False)

In [119]:
city_level = df[df['GEO_ID'].str.startswith('16')].to_dict('records')[0] # --> entire new york city estimates and margin of errors

In [120]:
df.loc[~df['GEO_ID'].str.startswith('16'), :] # --> all rows except city level 

,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,B01001_005E,B01001_005M,...,B03001_027E,B03001_027M,B03001_028E,B03001_028M,B03001_029E,B03001_029M,B03001_030E,B03001_030M,B03001_031E,B03001_031M
0,31611,-555555555,15700,61,711,25,843,84,777,85,...,68,33,31,23,13,14,0,24,24,20
1,1455846,-555555555,685636,195,55036,142,51974,1319,49368,1298,...,18154,1629,1698,383,1041,421,111,120,15304,1395
2,467669,-555555555,225685,101,13830,72,14245,516,14537,517,...,1833,321,649,223,412,229,0,27,772,209
3,53955,-555555555,26899,112,1474,59,1649,153,1626,168,...,172,111,24,24,39,33,0,27,109,104
4,81224,-555555555,41648,104,2056,79,2112,200,2062,196,...,287,83,71,57,4,6,0,27,212,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,110156,2105,49872,1306,3231,406,3187,414,3103,397,...,567,245,61,53,36,45,0,27,470,216
2281,165028,3738,78678,2396,5668,688,4796,608,4047,412,...,1750,507,620,337,30,40,0,27,1100,491
2282,244302,3220,114364,1921,8609,519,7039,586,8091,615,...,1434,334,129,83,170,119,0,27,1135,297
2283,138868,3101,67901,1953,4287,587,3429,481,4395,448,...,1174,357,286,130,95,79,0,27,793,277


### Formula:
https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf

In [132]:
# e --> estimate, m --> moe
def get_c(e, m): 
    if str(m).strip():
        return ''
    elif e == 0:
        return ''
    else: 
        return m/1.645/e*100

def get_p(e, agg_e):
    if agg_e == 0: 
        return ''
    else: 
        return e/agg_e*100

def get_z(e, m, p, agg_e, agg_m):
    if p == 0:
        return ''
    elif p == 100:
        return ''
    elif agg_e == 0:
        return ''
    elif m**2 - (e*agg_m/agg_e)**2 <0:
        return math.sqrt(m**2 + (e*agg_m/agg_e)**2)/agg_e*100
    else: 
        return math.sqrt(m**2 - (e*agg_m/agg_e)**2)/agg_e*100

variables = list(df.columns)
variables.remove('GEO_ID')
variables.remove('NAME')
var = list(set([i[:-1] for i in variables if i[:-1][-1] != 'P']))
for i in var:
    df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}C'] = df.apply(lambda row: get_c(row[f'{i}E'], row[f'{i}M']), axis=1)
    
    if f'{i}PE' in list(df.columns):
        df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}PE']
    else: 
        df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.apply(lambda row: get_p(row[f'{i}E'], 
                                                                                                city_level[f'{i}E']), axis=1)
    if f'{i}PM' in list(df.columns):
        df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}PM']
    else: 
        df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}Z'] = df.apply(lambda row: get_z(row[f'{i}E'], 
                                                                                                row[f'{i}M'], 
                                                                                                row[f'{i}P'], 
                                                                                                city_level[f'{i}E'], 
                                                                                                city_level[f'{i}M']), axis=1)

In [135]:
output_cols = [[i+'E', i+'M', i+'P', i+'Z'] for i in var]

In [143]:
output = ['GEO_ID', 'NAME']
for i in output_cols:
    output+=i

In [146]:
df[output].head()

,GEO_ID,NAME,B01001_005E,B01001_005M,B01001_005P,B01001_005Z,B01001_021E,B01001_021M,B01001_021P,B01001_021Z,...,B03001_018P,B03001_018Z,B02015_022E,B02015_022M,B02015_022P,B02015_022Z,B01001_002E,B01001_002M,B01001_002P,B01001_002Z
0,0500000US36095,"Schoharie County, New York",777,85,0.326306,0.0355118,593,99,0.646829,0.107396,...,0.000000,,5,8,0.034074,0.0544326,15700,61,0.384813,0.00149486
1,0500000US36005,"Bronx County, New York",49368,1298,20.732404,0.494068,13063,840,14.248784,0.881947,...,6.340909,3.86769,3044,597,20.744173,3.61888,685636,195,16.805187,0.00461258
2,0500000US36067,"Onondaga County, New York",14537,517,6.104905,0.206256,5893,341,6.427933,0.354678,...,0.681818,0.831182,2436,513,16.600791,3.16366,225685,101,5.531621,0.00244099
3,0500000US36035,"Fulton County, New York",1626,168,0.682849,0.0701438,864,136,0.942429,0.147433,...,0.000000,,0,27,0.000000,,26899,112,0.659304,0.00274472
4,0500000US36019,"Clinton County, New York",2062,196,0.865950,0.0817475,1040,150,1.134405,0.162417,...,0.000000,,13,18,0.088592,0.122409,41648,104,1.020808,0.00254794


In [145]:
df[['B01001_001E', 'B01001_001M', 'B01001_001C', 'B01001_001P', 'B01001_001Z']].head()

,B01001_001E,B01001_001M,B01001_001C,B01001_001P,B01001_001Z
0,31611,-555555555,,0.369284,6490.04
1,1455846,-555555555,,17.007404,6395.53
2,467669,-555555555,,5.463377,6480.39
3,53955,-555555555,,0.630310,6489.95
4,81224,-555555555,,0.948871,6489.79


#### Notes: 
for this following group, we do have e.g. Percent Margin of Error, maybe we shouldn't calculate these
```json
"DP05_0089PM": {
    "label": "Percent Margin of Error!!CITIZEN VOTING AGE POPULATION!!Citizen 18 and over population!!Female",
    "concept": "ACS DEMOGRAPHIC AND HOUSING ESTIMATES",
    "predicateType": "float",
    "group": "DP05",
    "limit": 0,
    "predicateOnly": true
},
```
https://api.census.gov/data/2017/acs/acs5/profile/groups/DP05.json